In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
path_df = "/content/drive/MyDrive/SDGP code/Classification/dataframes/full-dataframe.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [5]:
df.head()

,category,legName,pieceName,content
0,family,Rectification of a Muslim Marriage Register of...,0-Intro.txt,Rectification Of A Muslim Marriage Register Of...
1,family,Rectification of a Muslim Marriage Register of...,1-Short title.txt,This Act may be cited as the Rectification of ...
2,family,Rectification of a Muslim Marriage Register of...,2-Rectification of marriage register kept by a...,(1) Where any marriages have been registered b...
3,family,Kandyan Marriage and Divorce Act,0-Intro.txt,MARRIAGE AND DIVORCE (KANDYAN)\nAN ACT TO AMEN...
4,family,Kandyan Marriage and Divorce Act,1-Short title.txt,This Act may be cited as the Kandyan Marriage ...


In [6]:
df.loc[1]['content']

'This Act may be cited as the Rectification of a Muslim Marriage Register of the Paranakuru Korale (Kegalla District) Act, No. 4 of 1965.'

In [7]:
df['Content_Parsed_1'] = df['content'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

In [8]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

In [9]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

In [10]:
wordnet_lemmatizer = WordNetLemmatizer()

In [11]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)


In [12]:
df['Content_Parsed_5'] = lemmatized_text_list

In [13]:
stop_words = list(stopwords.words('english'))
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

In [14]:
df.loc[5]['Content_Parsed_6']

' provision   act shall  unless otherwise expressly provide therein apply  marriages contract   appoint date   '

In [15]:
df.head(1)

,category,legName,pieceName,content,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,family,Rectification of a Muslim Marriage Register of...,0-Intro.txt,Rectification Of A Muslim Marriage Register Of...,Rectification Of A Muslim Marriage Register Of...,rectification of a muslim marriage register of...,rectification of a muslim marriage register of...,rectification of a muslim marriage register of...,rectification of a muslim marriage register of...,rectification muslim marriage register par...


In [16]:
list_columns = ["category", "legName", "pieceName", "content", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [17]:
df.head()

,category,legName,pieceName,content,Content_Parsed
0,family,Rectification of a Muslim Marriage Register of...,0-Intro.txt,Rectification Of A Muslim Marriage Register Of...,rectification muslim marriage register par...
1,family,Rectification of a Muslim Marriage Register of...,1-Short title.txt,This Act may be cited as the Rectification of ...,act may cite rectification muslim marria...
2,family,Rectification of a Muslim Marriage Register of...,2-Rectification of marriage register kept by a...,(1) Where any marriages have been registered b...,(1) marriages register l aboosally mari...
3,family,Kandyan Marriage and Divorce Act,0-Intro.txt,MARRIAGE AND DIVORCE (KANDYAN)\nAN ACT TO AMEN...,marriage divorce (kandyan) act amend conso...
4,family,Kandyan Marriage and Divorce Act,1-Short title.txt,This Act may be cited as the Kandyan Marriage ...,act may cite kandyan marriage divorce act


In [18]:
category_codes = {
    'crime': 0,
    'family': 1,
    'rights': 2,
    'employment': 3
}

In [19]:
df['Category_Code'] = df['category']
df = df.replace({'Category_Code':category_codes})

In [20]:
df.head()

,category,legName,pieceName,content,Content_Parsed,Category_Code
0,family,Rectification of a Muslim Marriage Register of...,0-Intro.txt,Rectification Of A Muslim Marriage Register Of...,rectification muslim marriage register par...,1
1,family,Rectification of a Muslim Marriage Register of...,1-Short title.txt,This Act may be cited as the Rectification of ...,act may cite rectification muslim marria...,1
2,family,Rectification of a Muslim Marriage Register of...,2-Rectification of marriage register kept by a...,(1) Where any marriages have been registered b...,(1) marriages register l aboosally mari...,1
3,family,Kandyan Marriage and Divorce Act,0-Intro.txt,MARRIAGE AND DIVORCE (KANDYAN)\nAN ACT TO AMEN...,marriage divorce (kandyan) act amend conso...,1
4,family,Kandyan Marriage and Divorce Act,1-Short title.txt,This Act may be cited as the Kandyan Marriage ...,act may cite kandyan marriage divorce act,1


In [ ]:
# df.pop("pieceName")

0                                             0-Intro.txt
1                                       1-Short title.txt
2                                2-Application of Act.txt
3       3-Marriages between persons subject to Kandyan...
4                            4-Lawful age of marriage.txt
                              ...                        
1311    7-Community of goods not a consequence of marr...
1312    6-Rights of husband and wife in respect of imm...
1313    5-Rights of husband and wife domiciled or resi...
1314    4-This Ordinance not to affect rights acquired...
1315                                 3-Interpretation.txt
Name: pieceName, Length: 1316, dtype: object

In [21]:
df.head()

,category,legName,pieceName,content,Content_Parsed,Category_Code
0,family,Rectification of a Muslim Marriage Register of...,0-Intro.txt,Rectification Of A Muslim Marriage Register Of...,rectification muslim marriage register par...,1
1,family,Rectification of a Muslim Marriage Register of...,1-Short title.txt,This Act may be cited as the Rectification of ...,act may cite rectification muslim marria...,1
2,family,Rectification of a Muslim Marriage Register of...,2-Rectification of marriage register kept by a...,(1) Where any marriages have been registered b...,(1) marriages register l aboosally mari...,1
3,family,Kandyan Marriage and Divorce Act,0-Intro.txt,MARRIAGE AND DIVORCE (KANDYAN)\nAN ACT TO AMEN...,marriage divorce (kandyan) act amend conso...,1
4,family,Kandyan Marriage and Divorce Act,1-Short title.txt,This Act may be cited as the Kandyan Marriage ...,act may cite kandyan marriage divorce act,1


In [22]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], df['Category_Code'], test_size=0.15,random_state=8)

In [24]:
ngram_range = (1,3)
min_df = 5  
# max_df = 1.0
max_features = 300

In [25]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        # max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)
print(labels_train)

(1657, 300)
(293, 300)
595     1
141     1
1018    0
1494    3
1341    3
       ..
986     0
133     1
361     1
1364    3
451     1
Name: Category_Code, Length: 1657, dtype: int64


In [26]:
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'crime' category:
  . Most correlated unigrams:
. offence
. commit
. juvenile
. court
. young
  . Most correlated bigrams:
. young person
. child young

# 'employment' category:
  . Most correlated unigrams:
. contributor
. work
. convention
. labour
. pension
  . Most correlated bigrams:
. pension scheme
. international labour

# 'family' category:
  . Most correlated unigrams:
. centre
. kandyan
. district
. registrar
. marriage
  . Most correlated bigrams:
. registrar general
. district registrar

# 'rights' category:
  . Most correlated unigrams:
. party
. present
. state
. property
. right
  . Most correlated bigrams:
. young person
. district registrar



In [27]:
unigrams

['thereof',
 'apply',
 'mean',
 'general',
 'order',
 'women',
 'hold',
 'precede',
 'sum',
 'effect',
 'international',
 'attain',
 'ten',
 'shall',
 'every',
 'proceed',
 'due',
 'establish',
 'within',
 'competent',
 'members',
 'condition',
 'operation',
 'necessary',
 'force',
 'paragraph',
 'exercise',
 'immediately',
 'respect',
 'include',
 'report',
 'give',
 'day',
 'require',
 'duties',
 'committee',
 'discharge',
 'follow',
 'marriage',
 'receive',
 'upon',
 'thereto',
 'three',
 'unless',
 'first',
 'leave',
 'regulation',
 'benefit',
 'lawful',
 'place',
 'guardian',
 'either',
 'public',
 'bring',
 'rule',
 'appeal',
 'case',
 'enter',
 'undertake',
 'hereinafter',
 'information',
 'home',
 'applicant',
 'notice',
 'summary',
 'repeal',
 'adoption',
 'power',
 'time',
 'accordance',
 'grant',
 'procedure',
 'provide',
 'service',
 'without',
 'purpose',
 'contract',
 'consent',
 'contain',
 'ii',
 'form',
 'five',
 'two',
 'approval',
 'function',
 'aforesaid',
 'issue',

In [28]:
# X_train
with open('/content/drive/MyDrive/SDGP code/Classification/FeatureExtraction/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)

In [29]:
# X_test    
with open('/content/drive/MyDrive/SDGP code/Classification/FeatureExtraction/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('/content/drive/MyDrive/SDGP code/Classification/FeatureExtraction/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('/content/drive/MyDrive/SDGP code/Classification/FeatureExtraction/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('/content/drive/MyDrive/SDGP code/Classification/FeatureExtraction/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('/content/drive/MyDrive/SDGP code/Classification/FeatureExtraction/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('/content/drive/MyDrive/SDGP code/Classification/FeatureExtraction/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('/content/drive/MyDrive/SDGP code/Classification/FeatureExtraction/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('/content/drive/MyDrive/SDGP code/Classification/FeatureExtraction/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('/content/drive/MyDrive/SDGP code/Classification/FeatureExtraction/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)